# Building sub-pipelines

---

[$facet](https://docs.mongodb.com/manual/reference/operator/aggregation/facet/) - Processing multiple aggregation sub-pipelines within a single stage on the same set of input documents.

---
### Conneting to MongoDB using Pymongo
----

In [14]:
# Importing the required libraries
import pymongo
import pprint as pp

pp.sorted = lambda x, key=None: x

In [15]:
# Connect to the mongo client - Atlas Cluster
client = pymongo.MongoClient('mongodb://localhost:27017/')

In [16]:
# Database
db = client.training

In [17]:
# Sample document
pp.pprint(
    db.hr.find_one())

{'_id': ObjectId('60bc95fb12d1778df87722e2'),
 'enrollee_id': 23798,
 'gender': 'Male',
 'date_of_enrollment': datetime.datetime(2016, 8, 4, 8, 4, 14, 780000),
 'city': {'name': 'city_149', 'development_index': 0.689},
 'education': {'level': 'Graduate', 'discipline': 'STEM'},
 'experience': {'years': 3,
                'company_type': 'Pvt Ltd',
                'last_new_job': 1,
                'relevent_experience': 1},
 'training_hours': 106}


---
### Question - 

Find average training hours delivered per year and per month after 2017.

----

**Importing datetime library**

----

In [18]:
# Import library
from datetime import datetime

In [19]:
# Question
result = db.hr.aggregate(
                    [
                        # Stage 1 - Filter documents
                        {
                            '$match':{'date_of_enrollment':{'$gte':datetime(2018,1,1)}}
                        },
                        # Stage 2 - Project year
                        {
                            '$project':{
                                            'Year':{'$year':'$date_of_enrollment'},
                                            'training_hours':1
                                        }
                        },
                        # Stage 3 - Group by year
                        {
                            '$group':{
                                        '_id':'$Year',
                                        'avg_hours':{'$avg':'$training_hours'}
                                    }
                        },
                        # Stage 4 - Sort by avg_hours
                        {
                            '$sort':{'avg_hours':-1}
                        }
                    ]
                )

# Print result
for doc in result:
    pp.pprint(doc)

{'_id': 2018, 'avg_hours': 66.30190538764784}
{'_id': 2020, 'avg_hours': 65.10072178477691}
{'_id': 2019, 'avg_hours': 64.57091620476035}


In [20]:
# Question
result = db.hr.aggregate(
                    [
                        # Stage 1 - Filter documents
                        {
                            '$match':{'date_of_enrollment':{'$gte':datetime(2018,1,1)}}
                        },
                        # Stage 2 - Project month
                        {
                            '$project':{
                                            'Month':{'$month':'$date_of_enrollment'},
                                            'training_hours':1
                                        }
                        },
                        # Stage 3 - Group by year
                        {
                            '$group':{
                                        '_id':'$Month',
                                        'avg_hours':{'$avg':'$training_hours'}
                                    }
                        },
                        # Stage 4 - Sort by avg_hours
                        {
                            '$sort':{'avg_hours':-1}
                        }
                    ]
                )

# Print result
for doc in result:
    pp.pprint(doc)

{'_id': 7, 'avg_hours': 70.03066666666666}
{'_id': 8, 'avg_hours': 68.9}
{'_id': 9, 'avg_hours': 66.7911227154047}
{'_id': 5, 'avg_hours': 65.59628154050465}
{'_id': 3, 'avg_hours': 65.55583126550869}
{'_id': 11, 'avg_hours': 65.19695044472681}
{'_id': 2, 'avg_hours': 65.15492957746478}
{'_id': 1, 'avg_hours': 65.0608020698577}
{'_id': 6, 'avg_hours': 63.62234042553192}
{'_id': 12, 'avg_hours': 62.958015267175576}
{'_id': 10, 'avg_hours': 62.49484536082474}
{'_id': 4, 'avg_hours': 62.420833333333334}


---
We couldn't have answered the query with a single group stage.

----

In [21]:
# Question
result = db.hr.aggregate(
                    [
                        # Stage 1 - Filter documents
                        {
                            '$match':{'date_of_enrollment':{'$gte':datetime(2018,1,1)}}
                        },
                        # Stage 2 - Project month
                        {
                            '$project':{
                                            'Year':{'$year':'$date_of_enrollment'},
                                            'Month':{'$month':'$date_of_enrollment'},
                                            'training_hours':1
                                        }
                        },
                        # Stage 3 - Group by year
                        {
                            '$group':{
                                        '_id':{
                                                'Year':'$Year',
                                                'Month':'$Month'
                                                },
                                        'avg_hours':{'$avg':'$training_hours'}
                                    }
                        },
                        # Stage 4 - Sort by avg_hours
                        {
                            '$sort':{'avg_hours':-1}
                        }
                    ]
                )

# Print result
for doc in result:
    pp.pprint(doc)

{'_id': {'Year': 2018, 'Month': 8}, 'avg_hours': 73.93478260869566}
{'_id': {'Year': 2020, 'Month': 7}, 'avg_hours': 73.05714285714286}
{'_id': {'Year': 2018, 'Month': 1}, 'avg_hours': 70.83333333333333}
{'_id': {'Year': 2020, 'Month': 9}, 'avg_hours': 69.48809523809524}
{'_id': {'Year': 2019, 'Month': 7}, 'avg_hours': 69.05349794238683}
{'_id': {'Year': 2018, 'Month': 9}, 'avg_hours': 68.99615384615385}
{'_id': {'Year': 2018, 'Month': 4}, 'avg_hours': 68.97446808510638}
{'_id': {'Year': 2019, 'Month': 11}, 'avg_hours': 68.6491935483871}
{'_id': {'Year': 2018, 'Month': 7}, 'avg_hours': 68.10687022900764}
{'_id': {'Year': 2020, 'Month': 3}, 'avg_hours': 68.01532567049809}
{'_id': {'Year': 2019, 'Month': 12}, 'avg_hours': 67.72519083969466}
{'_id': {'Year': 2020, 'Month': 8}, 'avg_hours': 67.69201520912547}
{'_id': {'Year': 2019, 'Month': 5}, 'avg_hours': 67.64}
{'_id': {'Year': 2018, 'Month': 11}, 'avg_hours': 66.71217712177122}
{'_id': {'Year': 2020, 'Month': 5}, 'avg_hours': 66.281124

----
---
**To solve these questions we had to create two separate pipelines. This means we had to fetch the same documents twice from the database. Can we combine these two pipelines together and reduce the data we have to fetch from database?**

---
----

---

### Sub-pipelines using [$facet](https://docs.mongodb.com/manual/reference/operator/aggregation/facet/)

- `$facet` allows to create multiple aggregation sub-pipelines within a single stage.

- Each sub-pipeline can have multiple stages.

- Same documents are passed to all sub-pipelines from the preliminary stage to `$facet`.

- Output of one sub-pipeline within `$facet` is *not passed as input to the next sub-pipeline*.

- This way you would not have to fetch data from database for every aggregation.

- Useful for aggregating data along multiple dimensions.

----

**Syntax -**

{ $facet:

   {
   
      <outputField1>: [ <stage1>, <stage2>, ... ],
      
      <outputField2>: [ <stage1>, <stage2>, ... ],
      ...

   }
   
}

----

### Question -

Finding average training hours delivered per month and per year after 2017 using a single pipeline.


----

In [22]:
# Sub-pipeline

result = db.hr.aggregate(
    
    # Pipeline
    [
        # Stage 1 - Filter documents after 2017     
        {
            '$match':{'date_of_enrollment':{'$gte':datetime(2018,1,1)}}
        },
        
        # Stage 2 - Project year and month
        {
            '$project':{
                            'Year':{'$year':'$date_of_enrollment'},
                            'Month':{'$month':'$date_of_enrollment'},
                            'training_hours':1
                        }
        },
        
        # Stage 3 - Sub-pipelines
        {
            '$facet':{
                        # Sub-pipeline 1 - Aggregate along year
                        'Yearly_avg_hours':[
                                                # Stage 1 - Group
                                                {
                                                    '$group':{
                                                                '_id':'$Year',
                                                                'avg_hours':{'$avg':'$training_hours'}
                                                            }
                                                },
                                                # Stage 2 - Sort
                                                {
                                                    '$sort':{'avg_hours':-1}
                                                }
                                         ],
                
                        # Sub-pipeline 2 - Aggregate along month
                        'Monthly_avg_hours':[
                                                # Stage 1 - Group
                                                {
                                                    '$group':{
                                                                '_id':'$Month',
                                                                'avg_hours':{'$avg':'$training_hours'}
                                                            }
                                                },
                                                # Stage 2 - Sort
                                                {
                                                    '$sort':{'avg_hours':-1}
                                                }
                                         ] 
                        }
            }
    ])

# Print results
for doc in result:
    pp.pprint(doc)

{'Yearly_avg_hours': [{'_id': 2018, 'avg_hours': 66.30190538764784},
                      {'_id': 2020, 'avg_hours': 65.10072178477691},
                      {'_id': 2019, 'avg_hours': 64.57091620476035}],
 'Monthly_avg_hours': [{'_id': 7, 'avg_hours': 70.03066666666666},
                       {'_id': 8, 'avg_hours': 68.9},
                       {'_id': 9, 'avg_hours': 66.7911227154047},
                       {'_id': 5, 'avg_hours': 65.59628154050465},
                       {'_id': 3, 'avg_hours': 65.55583126550869},
                       {'_id': 11, 'avg_hours': 65.19695044472681},
                       {'_id': 2, 'avg_hours': 65.15492957746478},
                       {'_id': 1, 'avg_hours': 65.0608020698577},
                       {'_id': 6, 'avg_hours': 63.62234042553192},
                       {'_id': 12, 'avg_hours': 62.958015267175576},
                       {'_id': 10, 'avg_hours': 62.49484536082474},
                       {'_id': 4, 'avg_hours': 62.420833333333334}

---
### Question - 

What is average quarterly and half-yearly training hours delivered for enrollees after 2017?

-----

In [23]:
# Question

result = db.hr.aggregate(
    
    # Pipeline
    [
        # Stage 1 - Filter documents    
        {
            '$match':{'date_of_enrollment':{'$gte':datetime(2018,1,1)}}
        },
        
        # Stage 2 - Prooject fields
        {
            '$project':{
                            'Month':{'$month':'$date_of_enrollment'},
                            'training_hours':1
                        }
        },
        
        # Stage 3 - Sub-pipelines
        {
            '$facet':{
                        # Sub-pipeline 1 - Half-yearly result 
                        'Half_Yearly_avg_hours':[
                                                # Stage 1
                                                {
                                                    '$bucket':{
                                                                    'groupBy': '$Month',
                                                                    'boundaries':[1, 7,13],
                                                                    'output':{
                                                                                'avg_hours':{'$avg':'$training_hours'}
                                                                            }
                                                              }
                                                },
                                                # Stage 2
                                                {
                                                    '$sort':{'avg_hours':-1}
                                                }
                                             ],
                
                        # Sub-pipeline 2 - Quarterly result
                        'Quarterly_avg_hours':[
                                                # Stage 1
                                                {
                                                    '$bucket':{
                                                                    'groupBy': '$Month',
                                                                    'boundaries':[1, 4, 7, 10, 13],
                                                                    'output':{
                                                                                'avg_hours':{'$avg':'$training_hours'}
                                                                            }
                                                              }
                                                },
                                                # Stage 2
                                                {
                                                    '$sort':{'avg_hours':-1}
                                                }
                                             ] 
                        }
            }
    ])

# Print results
for doc in result:
    pp.pprint(doc)

{'Half_Yearly_avg_hours': [{'_id': 7, 'avg_hours': 66.0318622174381},
                           {'_id': 1, 'avg_hours': 64.59260079751883}],
 'Quarterly_avg_hours': [{'_id': 7, 'avg_hours': 68.56576655052265},
                         {'_id': 1, 'avg_hours': 65.26430755788554},
                         {'_id': 4, 'avg_hours': 63.90157303370786},
                         {'_id': 10, 'avg_hours': 63.55512984248617}]}


----
### Exercise - 

Find average training hours delivered per education discipline and per education level for enrollees who joined from 2016 ownwards using sub-pipelines.

----